In [ ]:
from env_game_theoretic import GameTheoreticEnv
from agent_policies_game_theoretic import QAgent, DQNAgent
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import random


In [ ]:

def export_to_csv_episode_data(states_per_step, filename='simulation_data.csv'):
    """
    Export episode data to CSV with one line per step, including for each agent:
    - current resource level
    - observation (emotion)
    - action selected
    - personal_reward
    - empathic_reward
    - combined_reward (total internal reward)
    """
    import csv

    # Number of agents
    n_agents = len(states_per_step[0]['observations'])

    # Dynamically construct CSV headers
    fieldnames = ['step', 'resource']
    for i in range(n_agents):
        fieldnames += [
            f'observation_{i}',
            f'action_{i}',
            f'personal_reward_{i}',
            f'empathic_reward_{i}',
            f'combined_reward_{i}'
        ]

    with open(filename, mode='w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for step_data in states_per_step:
            row = {'step': step_data['step'], 'resource': step_data.get('resource', None)}
            obs = step_data['observations']
            acts = step_data['actions']
            personal = step_data['personal_reward']
            empathic = step_data['empathic_reward']
            # combined reward stored as 'internal_total_reward'
            combined = step_data.get('combined_reward', [])

            for i in range(n_agents):
                row[f'observation_{i}'] = obs[i]
                row[f'action_{i}'] = acts[i]
                row[f'personal_reward_{i}'] = personal[i]
                row[f'empathic_reward_{i}'] = empathic[i]
                # safe fallback to 0 if missing
                row[f'combined_reward_{i}'] = combined[i] if i < len(combined) else None

            writer.writerow(row)

    return filename



In [ ]:

def plot_resource_evolution(states_per_step, env, save_path="resource_evolution.png"):
    """
    Generate a static image to visualize the evolution of the ressources
    """
    steps = [step['step'] for step in states_per_step]
    resources = [step['resource'] for step in states_per_step]

    plt.figure(figsize=(10, 6))
    plt.plot(steps, resources, label='Level of ressources', color='green', linewidth=2)
    plt.xlabel("Step")
    plt.ylabel("Ressource")
    plt.title(f"Fluctuation of ressources in the environment for agent: {agent_to_test} with empathy level: {alpha} and valuation of last meal: {beta}")
    plt.ylim(0, env.initial_resources * 1.1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()




In [ ]:
def save_q_table_detailed_to_csv(agents, filename="q_table_detailed.csv"):
    """
    Save each Q-value individually with action separation.
    CSV format: agent_id, state, action, expected_reward
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["agent_id", "state", "action", "expected_reward"])

        for agent_idx, agent in enumerate(agents):
            if hasattr(agent, 'q_table'):  # QAgent only
                for state, actions in agent.q_table.items():
                    for action, value in enumerate(actions):
                        writer.writerow([agent_idx, state, action, value])



In [ ]:

def visualize_q_table(filename):
    df = pd.read_csv(filename)
    agent_id = 0
    df_agent = df[df['agent_id'] == agent_id]

    pivot_table = df_agent.pivot(index='state', columns='action', values='expected_reward')

    plt.figure(figsize=(12, 6))
    plt.title(f"Q-table (agent {agent_id})")
    heatmap = plt.imshow(pivot_table.fillna(0), cmap='viridis', aspect='auto')
    plt.colorbar(heatmap, label='Expected Reward')
    plt.xlabel("Action")
    plt.ylabel("State")
    plt.xticks(ticks=range(len(pivot_table.columns)), labels=pivot_table.columns)
    plt.yticks(ticks=range(len(pivot_table.index)), labels=pivot_table.index)
    plt.tight_layout()
    plt.show()



In [ ]:

def filename_definer(agent_type,
                     episode_number,
                     emotion_type,
                     see_emotions,
                     alpha,
                     beta,
                     smoothing_type,
                     threshold_value,
                     emotion_rounder,
                     params_DQN,
                     params_QLearning):
    """
    name of the file order : 
    episode number
    agent_to_test = "DQN" or "QLearning"
    emotion_type = can be "average" or "vector"
    see_emotions = "False" or "True"
    alpha = 1  # parameter for the degree of empathy (the higher the value the higher the empathy in range 0 - 1)
    beta = 0.3 # valuation of the last meal
    smoothing_type = linear or sigmoid
    threshold_value  proportion of reward in the history necessary to have a positive emotion
    emotion_rounder = decimale of emotions

    the parameters of the agents are in the order :
Params_QL
    "learning_rate"
    "gamma"
    "epsilon"
    "epsilon_decay"
    "epsilon_min"

params_DQN =
    "learning_rate"
    "gamma"
    "epsilon"
    "epsilon_decay"
    "epsilon_min"
    "batch_size"
    "hidden_size"
    "update_target_every"

    return the filename of one episode with a random 6 int suffix
    """
    if agent_type == "DQN":
        params = params_DQN
        param_order = ["learning_rate", "gamma", "epsilon", "epsilon_decay", "epsilon_min", "batch_size", "hidden_size", "update_target_every"]
    elif agent_type == "QLearning":
        params = params_QLearning
        param_order = ["learning_rate", "gamma", "epsilon", "epsilon_decay", "epsilon_min"]
    else:
        raise ValueError(f"Unknown agent type: {agent_type!r}")

    # Ensure values appear in fixed order (no key names)
    param_values = "_".join(str(params[key]) for key in param_order)

    random_suffix = ''.join(str(random.randint(0, 9)) for _ in range(6))
    see_emotions_str = str(see_emotions)

    filename = (
        f"results_"
        f"{episode_number}_"
        f"{agent_type}_"
        f"{emotion_type}_"
        f"{see_emotions_str}_"
        f"{alpha}_"
        f"{beta}_"
        f"{smoothing_type}_"
        f"{threshold_value}_"
        f"{emotion_rounder}_"
        f"{param_values}_"
        f"{random_suffix}.csv"
    )

    return filename


if __name__ == '__main__':
    for episode_number in range(1, episodes+1):
        states, env, agents = run_simulation()
        filename_data = export_to_csv_episode_data(states,
                                                   filename=filename_definer(agent_type=agent_to_test,
                                                                             episode_number=episode_number,
                                                                             emotion_type=emotion_type,
                                                                             see_emotions=see_emotions,
                                                                             alpha=alpha,
                                                                             beta=beta,
                                                                             smoothing_type=smoothing_type,
                                                                             threshold_value=threshold_value,
                                                                             emotion_rounder=emotion_rounder,
                                                                             params_DQN=params_DQN,
                                                                             params_QLearning=params_QLearning
                                                                             )
                                                        )

        plot_resource_evolution(states,
                                env)
